In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashion-mnist-itba/submission_sample.csv
/kaggle/input/fashion-mnist-itba/fashion-mnist-itba-lab-ml-2018b.zip


In [2]:
!unzip /kaggle/input/fashion-mnist-itba/fashion-mnist-itba-lab-ml-2018b.zip

Archive:  /kaggle/input/fashion-mnist-itba/fashion-mnist-itba-lab-ml-2018b.zip
  inflating: train_labels.csv        
  inflating: test_images.npy         
  inflating: train_images.npy        


In [3]:
X = np.load('/kaggle/working/train_images.npy')
y = pd.read_csv('/kaggle/working/train_labels.csv')['label'].to_numpy()

X_test = np.load('/kaggle/working/test_images.npy')

In [4]:
X.shape, y.shape

((60000, 28, 28), (60000,))

In [5]:
y[:10]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

In [6]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader

from PIL import Image

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [8]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])


In [9]:
class CustomDataset(Dataset):
    def __init__(self, images, lbls, transform=None):
        self.transforms = transform
        self.imgs = images
        self.lbls = lbls

    def __len__(self):
        return self.lbls.shape[0]
    
    def __getitem__(self, idx):
        img = self.imgs[idx]
        img = np.repeat(img[..., np.newaxis], 3, axis=-1)
        img = Image.fromarray((img * 255).astype(np.uint8))

        label = self.lbls[idx]

        if self.transforms:
            img = self.transforms(img)
        
        return img, label

In [12]:
train_dataset = CustomDataset(
    X_train, y_train, transform = train_transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(
    X_val, y_val, transform = train_transform
)
val_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [29]:
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm

weights = ResNet50_Weights.IMAGENET1K_V2
model = resnet50(weights=weights)

num_classes = 10
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

'''
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
'''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {correct}/{total} = {correct/total:.4f}")

Epoch 1/10: 100%|██████████| 1500/1500 [01:04<00:00, 23.17it/s]


Epoch 1, Loss: 0.5894


Epoch 2/10: 100%|██████████| 1500/1500 [01:04<00:00, 23.18it/s]


Epoch 2, Loss: 0.3883


Epoch 3/10: 100%|██████████| 1500/1500 [01:04<00:00, 23.20it/s]


...

Epoch 4, Loss: 0.3157


Epoch 5/10: 100%|██████████| 1500/1500 [01:04<00:00, 23.15it/s]


Epoch 5, Loss: 0.2940


Epoch 6/10:   3%|▎         | 43/1500 [00:01<01:04, 22.69it/s]


KeyboardInterrupt: 

In [30]:
class CustomTestDataset(Dataset):
    def __init__(self, images, transform=None):
        self.transforms = transform
        self.imgs = images

    def __len__(self):
        return self.imgs.shape[0]
    
    def __getitem__(self, idx):
        img = self.imgs[idx]
        img = np.repeat(img[..., np.newaxis], 3, axis=-1)
        img = Image.fromarray((img * 255).astype(np.uint8))

        if self.transforms:
            img = self.transforms(img)
        
        return img

In [31]:
test_dataset = CustomTestDataset(X_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
all_preds = []

with torch.no_grad():
    for images in tqdm(test_loader):
        images = images.to(device)
        
        outputs = model(images)
        p = outputs.argmax(1)

        all_preds.extend(p.cpu().numpy())

100%|██████████| 313/313 [00:05<00:00, 61.51it/s]


In [33]:
all_preds[-10:]

[6, 1, 0, 4, 2, 3, 5, 2, 3, 7]

In [34]:
subm = pd.read_csv('/kaggle/input/fashion-mnist-itba/submission_sample.csv')
subm

,Category,Id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
9995,0,9995
9996,0,9996
9997,0,9997
9998,0,9998


In [35]:
subm['Category'] = all_preds
subm

,Category,Id
0,3,0
1,2,1
2,7,2
3,4,3
4,3,4
...,...,...
9995,3,9995
9996,5,9996
9997,2,9997
9998,3,9998


In [36]:
subm.to_csv('submfinal.csv', index=False)